In [1]:
from pathlib import Path
import csv
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors

PROJECT = Path(r"C:\Users\ashak\ad-quantum-repurpose")
RAW = PROJECT / "data" / "raw"
PROC = PROJECT / "data" / "processed"
LIG_RAW = RAW / "ligands"
LIG_PROC = PROC / "ligands"
LIG_PROC.mkdir(parents=True, exist_ok=True)

smiles_csv = LIG_RAW / "seed_smiles.csv"
assert smiles_csv.exists(), f"Missing {smiles_csv}"
ligands = []
with open(smiles_csv) as f:
    rdr = csv.DictReader(f)
    for row in rdr:
        ligands.append((row["name"].strip(), row["smiles"].strip()))

print("Ligands:", [n for n,_ in ligands])


Ligands: ['donepezil', 'memantine', 'rivastigmine', 'galantamine', 'selegiline', 'rasagiline', 'riluzole', 'fluoxetine', 'sertraline', 'nilvadipine']


In [2]:
from rdkit.Chem import rdDistGeom

def prep_ligand(name, smi, outdir):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        print("!! BAD SMILES for", name)
        return None
    mol = Chem.AddHs(mol)  # add hydrogens
    params = rdDistGeom.ETKDGv3()
    params.randomSeed = 0xf00d
    cid = AllChem.EmbedMolecule(mol, params)
    if cid == -1:
        print("!! Embed failed for", name)
        return None
    AllChem.MMFFOptimizeMolecule(mol, maxIters=500)

    sdf_path = outdir / f"{name}.sdf"
    pdb_path = outdir / f"{name}.pdb"
    w = Chem.SDWriter(str(sdf_path)); w.write(mol); w.close()
    Chem.MolToPDBFile(mol, str(pdb_path))
    return sdf_path, pdb_path

created = []
for name, smi in ligands:
    p = prep_ligand(name, smi, LIG_PROC)
    if p: created.append((name, *p))
print("Prepared ligands:", len(created))


Prepared ligands: 10


In [3]:
import shutil, subprocess
assert shutil.which("obabel"), "OpenBabel 'obabel' not found in PATH."

ok = 0
for name, sdf_path, pdb_path in created:
    pdbqt_path = LIG_PROC / f"{name}.pdbqt"
    # Use SDF as input (contains 3D + hydrogens); you could also use the PDB
    cmd = ["obabel", str(sdf_path), "-O", str(pdbqt_path)]
    r = subprocess.run(cmd)
    if r.returncode == 0 and pdbqt_path.exists():
        ok += 1
    else:
        print("!! PDBQT convert failed for", name)

print(f"PDBQT created: {ok} / {len(created)}")


PDBQT created: 10 / 10


In [4]:
rows = []
for name, _, _ in created:
    sdf = LIG_PROC / f"{name}.sdf"
    mol = next(Chem.SDMolSupplier(str(sdf), removeHs=False))
    mw = rdMolDescriptors.CalcExactMolWt(mol)
    psa = rdMolDescriptors.CalcTPSA(mol)
    clogp = rdMolDescriptors.CalcCrippenDescriptors(mol)[0]
    rows.append((name, mw, psa, clogp))
rows[:5]


[('donepezil', 392.20999275600076, 42.010000000000005, 3.8564000000000034),
 ('memantine', 181.183049736, 26.02, 3.084200000000001),
 ('rivastigmine', 221.10519334, 38.769999999999996, 2.0359),
 ('galantamine', 278.09429430800037, 39.44, 4.244900000000003),
 ('selegiline', 173.12044948, 12.03, 2.0361000000000002)]